In [1]:
# This notebook expects that Modin and Ray are installed, e.g. by `pip install modin[ray]`.
# Alternative way is to install everything via conda, like
#     conda create -n test-modin-intel -c intel/label/test -c conda-forge modin=0.8.intel jupyter
# then activate new environment and run jupyter from it

# NOTE: this is special version for showing cloud-cluster functionality.
# It requires installation of extra packages: `pip install cloudpickle rpyc`
# Also if your environment requires proxy for SSH you need to expose it via MODIN_SOCKS_PROXY environment variable,
# please note that it requires ray >= 0.8.7 to work
import modin.experimental.pandas as pd
from modin.experimental.cloud import create_cluster

Please note that some of these APIs deviate from pandas in order to provide improved performance.


In [2]:
columns_names = [
        "trip_id", "vendor_id", "pickup_datetime", "dropoff_datetime", "store_and_fwd_flag",
        "rate_code_id", "pickup_longitude", "pickup_latitude", "dropoff_longitude", "dropoff_latitude",
        "passenger_count", "trip_distance", "fare_amount", "extra", "mta_tax", "tip_amount",
        "tolls_amount", "ehail_fee", "improvement_surcharge", "total_amount", "payment_type",
        "trip_type", "pickup", "dropoff", "cab_type", "precipitation", "snow_depth", "snowfall",
        "max_temperature", "min_temperature", "average_wind_speed", "pickup_nyct2010_gid",
        "pickup_ctlabel", "pickup_borocode", "pickup_boroname", "pickup_ct2010",
        "pickup_boroct2010", "pickup_cdeligibil", "pickup_ntacode", "pickup_ntaname", "pickup_puma",
        "dropoff_nyct2010_gid", "dropoff_ctlabel", "dropoff_borocode", "dropoff_boroname",
        "dropoff_ct2010", "dropoff_boroct2010", "dropoff_cdeligibil", "dropoff_ntacode",
        "dropoff_ntaname", "dropoff_puma",
    ]
parse_dates=["pickup_datetime", "dropoff_datetime"]

In [3]:
with create_cluster('aws', '../../../aws_credentials',
                    cluster_name="rayscale-test",
                    region="eu-north-1", zone="eu-north-1b", image="ami-00e1e82d7d4ca80d3") as remote:
    df = pd.read_csv('https://modin-datasets.s3.amazonaws.com/trips_data.csv', names=columns_names,
                    header=None, parse_dates=parse_dates)

Cluster configuration valid.

Cluster: rayscale-test
Bootstraping AWS config
AWS config
  IAM Profile: ray-autoscaler-v1
  EC2 Key pair (head & workers): ray-autoscaler_4_eu-north-1
  VPC Subnets (head & workers): subnet-0995b6216dbfd237a
  EC2 Security groups (head & workers): sg-071a54090e55d92f3
  EC2 AMI (head & workers): ami-00e1e82d7d4ca80d3

No head node found. Launching a new cluster. Confirm [y/N]: y [automatic, due to --yes]

Acquiring an up-to-date head node
  Launching 1 nodes [subnet_id=subnet-0995b6216dbfd237a]
    Launched instance i-0fe38c76768f10015 [state=pending, info=pending]
  Launched a new head node
  Fetching the new head node
  
<1/1> Setting up head node
  Prepared bootstrap config
  New status: waiting-for-ssh
  [1/6] Waiting for SSH to become available
    Running `uptime` as a test.
    Waiting for IP
      Not yet available, retrying in 10 seconds
      Received: 13.48.10.50
    Running `uptime`
    Success.
  Updating cluster configuration. [hash=53d91b29

Exception in thread Thread-6:
Traceback (most recent call last):
  File "/localdisk/vnlitvin/miniconda3/envs/test-modin-intel/lib/python3.7/site-packages/ray/autoscaler/command_runner.py", line 385, in start_process
    self.process_runner.check_call(final_cmd)
  File "/localdisk/vnlitvin/miniconda3/envs/test-modin-intel/lib/python3.7/subprocess.py", line 363, in check_call
    raise CalledProcessError(retcode, cmd)
subprocess.CalledProcessError: Command '['ssh', '-tt', '-i', '/users/vnlitvin/.ssh/ray-autoscaler_4_eu-north-1.pem', '-o', 'StrictHostKeyChecking=no', '-o', 'UserKnownHostsFile=/dev/null', '-o', 'IdentitiesOnly=yes', '-o', 'ExitOnForwardFailure=yes', '-o', 'ServerAliveInterval=5', '-o', 'ServerAliveCountMax=3', '-o', 'ControlMaster=auto', '-o', 'ControlPath=/tmp/ray_ssh_3004486bfb/4e108a2563/%C', '-o', 'ControlPersist=10s', '-o', 'ProxyCommand=nc -x proxy-im.intel.com:1080 %h %p', '-o', 'ConnectTimeout=120s', 'ubuntu@13.48.10.50', 'bash', '--login', '-c', '-i', '\'true && s

  Failed to setup head node.


Cannot spawn cluster:
Traceback (most recent call last):
  File "/localdisk/vnlitvin/modin/modin/experimental/cloud/rayscale.py", line 162, in __do_spawn
    verbose=1,
  File "/localdisk/vnlitvin/miniconda3/envs/test-modin-intel/lib/python3.7/site-packages/ray/autoscaler/commands.py", line 178, in create_or_update_cluster
    override_cluster_name)
  File "/localdisk/vnlitvin/miniconda3/envs/test-modin-intel/lib/python3.7/site-packages/ray/autoscaler/commands.py", line 628, in get_or_create_head_node
    cli_logger.abort("Failed to setup head node.")
  File "/localdisk/vnlitvin/miniconda3/envs/test-modin-intel/lib/python3.7/site-packages/ray/autoscaler/cli_logger.py", line 343, in abort
    raise SilentClickException("Exiting due to cli_logger.abort()")
ray.autoscaler.cli_logger.SilentClickException: Exiting due to cli_logger.abort()



CannotSpawnCluster: cause: Exiting due to cli_logger.abort()
Cannot spawn cluster

In [4]:
remote, df

NameError: name 'remote' is not defined

In [5]:
def q1(df):
    return df.groupby("cab_type")["cab_type"].count()
def q2(df):
    return df.groupby("passenger_count", as_index=False).mean()[["passenger_count", "total_amount"]]
def q3(df):
    return df.groupby(["passenger_count", "pickup_datetime"]).size().reset_index()
def q4(df):
    transformed = pd.DataFrame({
        "passenger_count": df["passenger_count"],
        "pickup_datetime": df["pickup_datetime"].dt.year,
        "trip_distance": df["trip_distance"].astype("int64"),
    })
    return transformed.groupby(["passenger_count", "pickup_datetime", "trip_distance"])  \
            .size().reset_index().sort_values(by=["pickup_datetime", 0], ascending=[True, False])

In [6]:
q1(df)

10000

In [7]:
q2(df)

,passenger_count,total_amount
0,0,18.333333
1,1,15.258850
2,2,20.332356
3,3,13.748845
4,4,19.742688
5,5,14.786221
6,6,15.400085


In [8]:
q3(df)

,passenger_count,pickup_datetime,0
0,0,2013-08-14 12:07:00,1
1,0,2013-08-14 12:37:00,1
2,0,2013-08-15 00:00:00,1
3,1,2013-08-01 08:14:37,1
4,1,2013-08-01 09:48:00,1
...,...,...,...
9909,6,2013-09-28 18:30:15,1
9910,6,2013-09-28 19:57:22,1
9911,6,2013-09-29 18:47:29,1
9912,6,2013-09-30 02:27:33,1


In [9]:
q4(df)

,passenger_count,pickup_datetime,trip_distance,0
2,1,2013,0,1991
3,1,2013,1,1270
4,1,2013,2,853
80,5,2013,0,551
81,5,2013,1,537
...,...,...,...,...
77,4,2013,10,1
78,4,2013,11,1
79,4,2013,14,1
102,5,2013,28,1


In [ ]:
%timeit q1(df)

In [ ]:
%timeit q2(df)

In [ ]:
%timeit q3(df)

In [ ]:
%timeit q4(df)